In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, BatchNormalization, MaxPooling2D, Conv2D
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras import backend as K
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess_input, decode_predictions as vgg16_decode_predictions
from keras.optimizers import SGD, RMSprop, Adam

In [ ]:
opt = RMSprop(lr=0.001, rho=0.7)
for layer in layers: layer.trainable=False
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
data_dir = 'data/dogscats/'
train_data_dir = f'{data_dir}train'
valid_data_dir = f'{data_dir}valid'
img_size = 224
batch_size = 64

train_datagen = ImageDataGenerator(
    preprocessing_function = vgg16_preprocess_input,
    rotation_range = 9,
    zoom_range = 0.2,
    shear_range = 0.2
)

valid_datagen = ImageDataGenerator(preprocessing_function = vgg16_preprocess_input )

train_generator = train_datagen.flow_from_directory(
    train_data_dir, 
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir, 
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
model.fit_generator(
    generator=train_generator,
    epochs=2,
    steps_per_epoch=train_generator.n // batch_size,
    workers = 8,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // batch_size
)

In [ ]:
base_model = VGG16(include_top=True, weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
layers = base_model.layers
last_conv_id = [index for index, layer in enumerate(layers) 
                if isinstance(layer, Conv2D)][-1]
conv_layers = layers[:last_conv_id+1]
fc_layers = layers[last_conv_id+1:]

In [ ]:
model = Sequential(conv_layers)

In [ ]:
def copy_weights(source_layers, target_layers):
    source_dense_ids = [idx for idx, layer in enumerate(source_layers)
                       if isinstance(layer, Dense)]
    target_dense_ids = [idx for idx, layer in enumerate(target_layers)
                       if isinstance(layer, Dense)]
    
    for i, idx in enumerate(source_dense_ids):
        target_layers[target_dense_ids[i]].set_weights(source_layers[idx].get_weights())

def get_top_layers(dropout, input_shape, fc_layers):
    top_model = Sequential([
        MaxPooling2D(input_shape=input_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(dropout),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(dropout),
        Dense(2, activation='softmax')
    ])
    copy_weights(fc_layers, top_model.layers)
    return top_model

In [ ]:
top_model = get_top_layers(0.6, conv_layers[-1].output_shape[1:], fc_layers)

In [ ]:
for l in model.layers: l.trainable = False
for l in top_model.layers: model.add(l)
model.summary()

In [ ]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, epochs=3, steps_per_epoch=train_generator.n // batch_size,
                    workers = 8, validation_data=valid_generator, validation_steps=valid_generator.n // batch_size)

In [ ]:
model.fit_generator(train_generator, epochs=3, steps_per_epoch=train_generator.n // batch_size,
                    workers = 8, validation_data=valid_generator, validation_steps=valid_generator.n // batch_size)

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, epochs=8, steps_per_epoch=train_generator.n // batch_size,
                    workers = 8, validation_data=valid_generator, validation_steps=valid_generator.n // batch_size)